In [16]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
import collections
from tqdm import tqdm
 
from tensorflow.keras.layers import Dropout, GRU, Embedding, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop 

In [3]:
X = np.load('./data/X_train.npy', allow_pickle=True)
y = np.load('./data/y_train.npy', allow_pickle=True)

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5)

In [7]:
data_train = []
maxx = 0
for data in X_train:
    if len(data) > maxx:
        maxx = len(data)
print(maxx)

108


In [43]:
def create_index(input_text):
    # TODO#2:edit or replace this function

    # # Create threshold to divide rare word from the chuck of input text
    word_counter = collections.Counter(input_text)
    words = list()
    # THRESH = int(0.001 * len(set(input_text)))
    THRESH = 1
    for word in tqdm(input_text):
        if word_counter[word] >= THRESH:
            words.append(word)

    # words = [word for word in input_text ]
    word_count = list()
    #use set and len to get the number of unique words
    word_count.extend(collections.Counter(words).most_common(len(set(words))))
    #include a token for unknown word
    word_count.append(("UNK",0))
    #print out 10 most frequent words
    print(word_count[:20])
    dictionary = dict()

#     dictionary["for_keras_zero_padding"] = 0
    
    for word in word_count:
        dictionary[word[0]] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    data = list()
    for word in input_text:
        if word not in dictionary:
            data.append(dictionary['UNK'])
        else:
            data.append(dictionary[word])

    

    return data,dictionary, reverse_dictionary

def encode_to_feed(chunk_data, dictionary):
    result = []
    for sentence in chunk_data:
        tmp = []
        for word in sentence:
            if word not in dictionary:
                tmp.append(dictionary['UNK'])
            else:
                tmp.append(dictionary[word])
        result.append(tmp)
    return result


In [53]:
maxlen = 200


# flatten to create a dictionary
all_train_words = set()
for ls in X_train:
    tmp = []
    for data in ls:
        all_train_words.add(data)
        tmp.append(data)
    tmp = np.array(tmp)
    
    
dataset,dictionary, reverse_dictionary=create_index(all_train_words)
    
encoded_X_train = encode_to_feed(X_train, dictionary)
encoded_X_val = encode_to_feed(X_val, dictionary)
encoded_X_test = encode_to_feed(X_test, dictionary)

data_train = pad_sequences(encoded_X_train, maxlen=maxlen, padding='post')
data_val = pad_sequences(encoded_X_val, maxlen=maxlen, padding='post')
data_test = pad_sequences(encoded_X_test, maxlen=maxlen, padding='post')

100%|██████████| 1397/1397 [00:00<00:00, 2873684.50it/s]

[('', 1), ('ค่', 1), ('ลุง', 1), ('จง', 1), ('9', 1), ('เจ้าตัว', 1), ('อธิบาย', 1), ('จิง', 1), ('รุ้จัก', 1), ('สัดส่วน', 1), ('ฝน', 1), ('ใบ้', 1), ('เทคนิค', 1), ('ค่เ', 1), ('ATM', 1), ('พื้นที่', 1), ('ทำไม', 1), ('โปรโมชั่น', 1), ('อาจ', 1), ('เข้า', 1)]


1183

In [42]:
encoded_X_train[1]

[552, 1382, 86, 943, 1322, 666, 1242, 1157, 724, 870]

In [31]:
len(data_train)

5996